# 📊 Teoría de Medias Móviles

Las **Medias Móviles (Moving Averages, MA)** son indicadores técnicos que suavizan las series de precios para identificar la **tendencia** subyacente.  
Su objetivo es reducir el "ruido" de los movimientos diarios y facilitar la detección de patrones.

---

## 🔹 Media Móvil Simple (SMA)

La **SMA** se calcula como el promedio aritmético de los precios de cierre en los últimos $n$ períodos:

$$
SMA_t = \frac{1}{n} \sum_{i=0}^{n-1} P_{t-i}
$$

Donde:  
- $P_t$ = precio en el tiempo $t$  
- $n$ = número de períodos (ej. 20, 50, 200)  

**Interpretación:**  
- Si el precio está **por encima** de la SMA → tendencia alcista.  
- Si el precio está **por debajo** de la SMA → tendencia bajista.  

---

## 🔹 Media Móvil Exponencial (EMA)

La **EMA** da mayor peso a los precios recientes, reaccionando más rápido a cambios de tendencia:

$$
EMA_t = \alpha P_t + (1 - \alpha) EMA_{t-1}
$$

con

$$
\alpha = \frac{2}{n+1}
$$

---

## 🔹 Comparación SMA vs EMA
- **SMA**: más suave, lenta en reaccionar.  
- **EMA**: más sensible, útil para detectar giros rápidos.  

---

## 🔹 Cruces de Medias Móviles

Una estrategia clásica consiste en usar dos medias móviles:  
- Una **rápida** (ej. 10 períodos).  
- Una **lenta** (ej. 50 períodos).  

- **Golden Cross** (cruce dorado):  
  Cuando la media móvil rápida cruza **por encima** de la lenta → señal alcista.  

  $$
  EMA_{rápida}(t) > EMA_{lenta}(t)
  $$

- **Death Cross** (cruce de la muerte):  
  Cuando la media móvil rápida cruza **por debajo** de la lenta → señal bajista.  

  $$
  EMA_{rápida}(t) < EMA_{lenta}(t)
  $$

---

## 📊 Interpretación práctica
- Se usan para **detectar tendencias** y como niveles de **soporte/resistencia dinámicos**.  
- Cuanto mayor el período, **más confiable** es la señal (pero más tardía).  
- Se combinan con otros indicadores (ej. MACD, RSI) para confirmar señales.  

In [1]:
# @title
!pip install yfinance plotly
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from dateutil.relativedelta import relativedelta


!pip install jupyter nbformat ipykernel ipywidgets


In [2]:
# @title
def _to_num(s):
    return pd.to_numeric(s, errors="coerce")

def _fmt2(x):
    try:
        x = float(x)
        return f"{x:.2f}"
    except Exception:
        return "NA"

def plot_interactive_candlestick_ohlc(
    ticker, start_date, end_date,
    ma_fast=10, ma_slow=20, ma_corr=100,
    width=1250, height=900
):
    data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    if data.empty:
        print(f"No data found for {ticker} between {start_date} and {end_date}")
        return

    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if col in data.columns:
            data[col] = _to_num(data[col])

    data['MA_Fast'] = data['Close'].rolling(window=ma_fast).mean()
    data['MA_Slow'] = data['Close'].rolling(window=ma_slow).mean()
    data['MA_Corr'] = data['Close'].rolling(window=ma_corr).mean()

    hovertext = [
        f"<b>{pd.to_datetime(d).date()}</b><br>"
        f"O:{_fmt2(o)} H:{_fmt2(h)} L:{_fmt2(l)} C:{_fmt2(c)}"
        for d, o, h, l, c in zip(data.index, data['Open'], data['High'], data['Low'], data['Close'])
    ]

    fig = make_subplots(rows=1, cols=1)

    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'],
        name="Candlestick",
        increasing_line_color="#26a69a", decreasing_line_color="#ef5350",
        hovertext=hovertext, hoverinfo="text+x+y",
        visible=True
    ))

    fig.add_trace(go.Ohlc(
        x=data.index,
        open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'],
        name="OHLC",
        increasing_line_color="#26a69a", decreasing_line_color="#ef5350",
        hovertext=hovertext, hoverinfo="text+x+y",
        visible=False
    ))

    fig.add_trace(go.Scatter(
        x=data.index, y=data['MA_Fast'], mode="lines", name=f"SMA {ma_fast}",
        line=dict(color="goldenrod", width=2.2),
        hovertemplate="SMA Rápida: %{y:.2f}<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=data.index, y=data['MA_Slow'], mode="lines", name=f"SMA {ma_slow}",
        line=dict(color="blue", width=2.2),
        hovertemplate="SMA Lenta: %{y:.2f}<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=data.index, y=data['MA_Corr'], mode="lines", name=f"SMA {ma_corr}",
        line=dict(color="purple", width=2.2),
        hovertemplate="SMA Corr: %{y:.2f}<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=data.index, y=data['Close'], mode="lines", name="Close",
        line=dict(width=1.3, dash="dot"), opacity=0.6,
        hovertemplate="Close: %{y:.2f}<extra></extra>"
    ))

    # Botones arriba izquierda
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            direction="right",
            x=0.5, y=1.35,                # 👈 coordenadas normalizadas
            xanchor="left", yanchor="top",
            buttons=[
                dict(label="Candlestick", method="update",
                     args=[{"visible": [True, False, True, True, True]}]),
                dict(label="OHLC", method="update",
                     args=[{"visible": [False, True, True, True, True]}]),
            ],
            pad=dict(r=8, t=8, b=8, l=8),
            bgcolor="rgba(240,240,240,0.9)"
        )]
    )

    fig.update_layout(
        title=f"{ticker} — Velas / OHLC con MAs ({start_date} a {end_date})",
        hovermode="x unified",
        xaxis=dict(
            rangeslider=dict(visible=True),
            type="date",
            rangeselector=dict(
                y=1.08, yanchor="top",
                buttons=[
                    dict(count=1, step="month", stepmode="backward", label="1M"),
                    dict(count=3, step="month", stepmode="backward", label="3M"),
                    dict(count=6, step="month", stepmode="backward", label="6M"),
                    dict(step="all", label="All")
                ]
            )
        ),
        yaxis_title="Precio",
        legend=dict(
            orientation="h",
            yanchor="bottom", y=1.35,
            xanchor="center", x=0.5
        ),
        width=width, height=height,
        margin=dict(t=120, r=20, b=60, l=60)
    )

    fig.show()


In [3]:
#@title Seleccione Acción y Medias Móviles {run:'auto'}
ticker_symbol = "VZ" #@param ["AAPL", "PLTR", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "TSM","BRK.B", "V", "JPM", "XOM", "LLY", "MRK", "UNH", "PG", "MA","CVX", "KO", "PEP", "COST", "TMO", "ORCL", "CSCO", "NKE", "VZ", "ASML", "TXN", "ABT", "TM", "SAP", "AMD", "NFLX", "NOW", "ADBE", "LVMUY", "BABA", "SHEL", "TMUS", "QCOM", "PFE", "SNY", "AZN", "TOT", "GSK", "RIO", "BHP", "MCD​"]
#ticker_symbol = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]

end_date   =  datetime.today().strftime("%Y-%m-%d")
start_date = (datetime.today() - relativedelta(years=1)).strftime("%Y-%m-%d")
MARapida = 10 #@param {type:"integer"}
MALenta  = 20 #@param {type:"integer"}
MACorr   = 100 #@param {type:"integer"}

In [4]:
# Uso
plot_interactive_candlestick_ohlc(
    ticker_symbol, start_date, end_date,
    ma_fast=MARapida, ma_slow=MALenta, ma_corr=MACorr,
)

In [5]:
# 3 Cómo calcular las ganancias con esta estrategia
inversion_inicial = 100_000.00 # usd
# Compras
# 1
golden_cross1 = 39    # Solo cambia el valor aquí
acciones1 = inversion_inicial / golden_cross1
death_cross1 = 41.5  # Solo cambia el valor aquí
ganancia1 = acciones1 * death_cross1
#
print(f"Inversion Inicial {inversion_inicial:,.2f}")
print(f"Acciones Compradas {acciones1:,.2f}")
print(f"Golden Cross {golden_cross1:,.2f}")
print(f"Death Cross {death_cross1:,.2f}")
print(f"Ganancia {ganancia1:,.2f}")


Inversion Inicial 100,000.00
Acciones Compradas 2,564.10
Golden Cross 39.00
Death Cross 41.50
Ganancia 106,410.26


In [6]:
# 2
golden_cross2 = 37.7        # Solo cambia el valor aquí
acciones2 = ganancia1 / golden_cross2
death_cross2 = 42.7     # Solo cambia el valor aquí
ganancia2 = acciones2 * death_cross2
#
print(f"Acciones Compradas {acciones2:,.2f}")
print(f"Golden Cross {golden_cross2:,.2f}")
print(f"Death Cross {death_cross2:,.2f}")
print(f"Ganancia {ganancia2:,.2f}")

Acciones Compradas 2,822.55
Golden Cross 37.70
Death Cross 42.70
Ganancia 120,523.02


In [7]:
# 3
golden_cross3 = 42.2        # Solo cambia el valor aquí
acciones3 = ganancia2 / golden_cross3
death_cross3 = 44.2     # Solo cambia el valor aquí
ganancia3 = acciones3 * death_cross3
#
print(f"Acciones Compradas {acciones3:,.2f}")
print(f"Golden Cross {golden_cross3:,.2f}")
print(f"Death Cross {death_cross3:,.2f}")
print(f"Ganancia {ganancia3:,.2f}")

Acciones Compradas 2,856.00
Golden Cross 42.20
Death Cross 44.20
Ganancia 126,235.01


In [8]:
# Reporte Final
rendimiento = (ganancia3 - inversion_inicial) / inversion_inicial
print(f"Ganancia Final {ganancia3:,.2f}")
print(f"Rendimiento {rendimiento:,.2%}")

Ganancia Final 126,235.01
Rendimiento 26.24%


# TO-DO:
- Arreglar el GUI para verlo en VSCode
- Identificar de forma automatica Golden Cross >45 grados de diferencia con la linea azul (EMA 20)